# Load GEMD files in database
This code is for information purposes onlny as it requires permissions on our SciServer system that users of this repo will not in general posess. The database schema is defined by files in the database_design subfolder.

#### NB parts of this code can only be run by someone with direct credentials to the database

In [ ]:
import glob
import json
import random
import pandas
import pprint
import sys
import json
import numpy as np
import pymssql

In [ ]:
# import sqlalchemy as sqla
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
# from sqlalchemy import select

In [ ]:
from gemd_database import MSSQLDatabase

In [ ]:
with open('/home/idies/workspace/persistent/.private/gemd.json','r') as f:
    AUTH = json.load(f)
GEMDDB=MSSQLDatabase(AUTH,'GEMD')

# Load in GEMD database
Prepared with 
```sql
create table GEMDModel (
    id bigint identity(1,1) not null
  , name varchar(1024) not null
  , constraint pk_gemdmodel primary key(id)
)

create table GEMDContext (
  uid varchar(64) not null
  , model_id bigint not null
  , gemd_type varchar(32) not null
  , context varchar(max) not null
  , constraint pk_GEMDContext primary key(uid)
  , constraint fk_gemdcontext_model foreign key (model_id) REFERENCES gemdmodel(id)
  )
```

In [ ]:
ddl = """
create table GEMDModel (
    id bigint identity(1,1) not null
  , name varchar(1024) not null
  , constraint pk_gemdmodel primary key(id)
)

create table GEMDObject (
  uid varchar(64) not null
  , model_id bigint not null
  , gemd_type varchar(32) not null
  , context varchar(max) not null
  , constraint pk_GEMDContext primary key(uid)
  , constraint fk_gemdcontext_model foreign key (model_id) REFERENCES gemdmodel(id)
)
  
CREATE TABLE [dbo].[GEMDAttribute](
  [gemdobject_uid] [varchar](64) NOT NULL,
  [template_id] [nvarchar](128) NULL,
  [name] [nvarchar](128) NULL,
  [value] [nvarchar](4000) NULL,
  [value_type] [nvarchar](32) NULL,
  [value_units] [nvarchar](128) NULL,
  [attribute_type] [nvarchar](128) NULL,
  [property] [nvarchar](max) NULL,
  [conditions] [nvarchar](max) NULL
)

 CREATE TABLE GEMDEdge(
    id bigint IDENTITY(1,1)  not null
  , from_uid varchar(64) not null
  , to_uid varchar(64) not null
  , gemd_ref varchar(64) not null
)
"""
# GEMDDB.execute_update(ddl)

In [ ]:
Base = automap_base()
# Base.prepare(GEMDDB.ENGINE, reflect=True)
Base.prepare(GEMDDB.ENGINE)#, reflect=True)
for c in Base.classes:
    print(c)
GEMDObject = Base.classes.GEMDObject
GEMDModel  = Base.classes.GEMDModel

## create object files from the JSON GEMD 
complete_data_model.json is located in an aptly named folder and contains all JSON content to be loaded

In [ ]:
folders = glob.glob('/home/idies/workspace/Temporary/gerard/GEMD-Maggie/GEMD_models/*')

In [ ]:
%%time
models = []
for folder in folders:
    model = GEMDModel(name=folder.split("/")[-1])
    models.append(model)
    files = glob.glob(f"{folder}/*")
    for file in files:
        with open(f'{file}','r') as f:
            try:
                d=json.load(f)
                GEMDObject(gemd_type=d['type'],uid=d['uids']['auto'],context=json.dumps(d),gemdmodel=model)
            except Exception as e:
                print("ERROR:",file)
                print(e)

In [ ]:
%%time
session = Session(GEMDDB.ENGINE)
for model in models:
    session.add(model)
session.commit()

## extract attributes

In [ ]:
sql="""
with a as (
select s.uid, s.gemd_type
,      sp.*
  from GEMDObject s
  cross apply OPENJSON(s.context,'$.parameters') with (
     template_id nvarchar(128) '$.template.id',
     name nvarchar(128) '$.name',
     value_type nvarchar(max) '$.value.type' ,
     value_units nvarchar(max) '$.value.units' ,
     value nvarchar(max) '$.value' as json,
     attribute_type nvarchar(128) '$.type',
     parameter nvarchar(max) '$' as json) sp
)
insert into GEMDAttribute
select a.uid as gemdcontext_uid, a.template_id, a.name
,      case when a.value_type = 'nominal_categorical' then json_value(a.value,'$.category')
            when a.value_type = 'nominal_real' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_integer' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_composition' then json_value(a.value,'$')
       else NULL end as value
       , a.value_type
       , a.value_units
,      a.attribute_type
,      a.parameter, NULL
  from a 
"""
GEMDDB.execute_update(sql)

In [ ]:
sql = """
with c as (
select s.uid, s.gemd_type, sp.property_type
,      case when sp.property_type = 'property_and_conditions' then sp.p_property
            else sp.property end property
,      case when sp.property_type = 'property_and_conditions' then sp.p_conditions
            else NULL end conditions
  from GEMDObject s
  cross apply OPENJSON(s.context,'$.properties') with (
     property_type nvarchar(128) '$.type',
     p_property nvarchar(max) '$.property' as json,
     p_conditions nvarchar(max) '$.conditions' as json,
     property nvarchar(max) '$' as json) sp
), a as (
select c.uid,c.gemd_type, c.property_type, c.property
,      case when c.conditions = '[]' then NULL else c.conditions end as conditions
,      sp.*
  from c
  cross apply OPENJSON(c.property,'$') with (
     template_id nvarchar(128) '$.template.id',
     name nvarchar(128) '$.name',
     value_type nvarchar(max) '$.value.type' ,
     value_units nvarchar(max) '$.value.units' ,
     value nvarchar(max) '$.value' as json) sp
)
insert into GEMDAttribute
select a.uid as gemdcontext_uid, a.template_id, a.name
,      case when a.value_type = 'nominal_categorical' then json_value(a.value,'$.category')
            when a.value_type = 'nominal_real' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_integer' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_composition' then json_value(a.value,'$')
       else NULL end as value
,      a.value_type
,      a.value_units
,      a.property_type as attribute_type
,      a.property
,      a.conditions
  from a 
"""
GEMDDB.execute_update(sql)

In [ ]:
sql = """
with a as (
select s.uid, s.gemd_type
,      sp.*
  from GEMDObject s
  cross apply OPENJSON(s.context,'$.conditions') with (
     template_id nvarchar(128) '$.template.id',
     name nvarchar(128) '$.name',
     value_type nvarchar(max) '$.value.type' ,
     value_units nvarchar(max) '$.value.units' ,
     value nvarchar(max) '$.value' as json,
     attribute_type nvarchar(128) '$.type',
     condition nvarchar(max) '$' as json) sp
)
insert into GEMDAttribute
select a.uid as gemdcontext_uid, a.template_id, a.name
,      case when a.value_type = 'nominal_categorical' then json_value(a.value,'$.category')
            when a.value_type = 'nominal_real' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_integer' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_composition' then json_value(a.value,'$')
       else NULL end as value
       , a.value_type
       , a.value_units
,      a.attribute_type
,      a.condition, NULL
  from a 
"""
GEMDDB.execute_update(sql)

# utility views

GEMDObjectAttributes: GEMD objects with attributes (properties, parameters, conditions) aggregated in JSON string

In [ ]:
sql = """
create or alter view GEMDObjectAttributes as
select o.model_id,o.uid, o.gemd_type
,      '{'+STRING_AGG('"'+a.name+'":"'+cast(a.value as nvarchar(max))+'"',',') +'}' as attributes
from [dbo].[GEMDObject] o
left outer join gemdattribute a on a.gemdobject_uid=o.uid
group by o.model_id, o.uid, o.gemd_type
"""
GEMDDB.execute_update(sql)

alternative: name/value pairs

In [ ]:
sql = """
create or alter view GEMDObjectAttributesJ as
select o.model_id, o.uid, o.gemd_type
,      (select name, value, value_units, attribute_type 
          from gemdattribute a 
         where a.gemdobject_uid=o.uid
           for json path) as attributes
from [dbo].[GEMDObject] o
group by o.model_id, o.uid, gemd_type
"""
GEMDDB.execute_update(sql)